In [ ]:
import os

REPO = "SolarFlareZ/aml-project"
BRANCH = "main"
PROJECT_DIR = "/content/aml-project"

if os.path.exists(PROJECT_DIR):
    print("repo exists, pulling last commit")
    %cd {PROJECT_DIR}
    !git checkout {BRANCH}
    !git pull origin {BRANCH}
else:
    print("cloning repo...")
    !git clone -b {BRANCH} https://github.com/{REPO}.git
    %cd {PROJECT_DIR}

!pip install -q -r requirements.txt

print(f"\ncurr dir: {os.getcwd()}")

In [ ]:
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

DRIVE_RESULTS = "/content/drive/MyDrive/aml-project-results"
LOCAL_RESULTS = f"{PROJECT_DIR}/results"

os.makedirs(DRIVE_RESULTS, exist_ok=True)

if os.path.islink(LOCAL_RESULTS):
    os.unlink(LOCAL_RESULTS)
elif os.path.exists(LOCAL_RESULTS):
    shutil.rmtree(LOCAL_RESULTS)

os.symlink(DRIVE_RESULTS, LOCAL_RESULTS)

print(f"Symlinked {LOCAL_RESULTS} to {DRIVE_RESULTS}")

In [ ]:
!python -m src.train_fedavg \
    federated.num_clients=10 \
    federated.num_rounds=5 \
    federated.participation_rate=0.5 \
    logging.eval_every=1 \
    experiment_name=fedavg_quick_test

In [ ]:
# IID
for j in [4, 8, 16]:
    num_rounds = 500 // (j // 4) # might be bad... Can find specific instead
    
    print(f"Running: J={j}, rounds={num_rounds}")
    
    !python -m src.train_fedavg \
        federated.num_clients=100 \
        federated.num_rounds={num_rounds} \
        federated.participation_rate=0.1 \
        federated.local_steps={j} \
        federated.sharding=iid \
        logging.eval_every=10 \
        experiment_name=fedavg_iid_j{j}

In [ ]:
# non iid
for nc in [1, 5, 10, 50]:
    for j in [4, 8, 16]:
        num_rounds = 500 // (j // 4) # 500, 250, 125. might be bad... Can find specific instead
        
        print(f"Running: Nc={nc}, J={j}, rounds={num_rounds}")
        
        !python -m src.train_fedavg \
            federated.num_clients=100 \
            federated.num_rounds={num_rounds} \
            federated.participation_rate=0.1 \
            federated.local_steps={j} \
            federated.sharding=non_iid \
            federated.num_classes_per_client={nc} \
            logging.eval_every=10 \
            experiment_name=fedavg_nc{nc}_j{j}